In [74]:
from bevy_env import BevyRLEnv
from tqdm import tqdm

In [75]:
env = BevyRLEnv()

In [76]:
import matplotlib.pyplot as plt
import numpy as np

from quadcopter_control.controller import QuadcopterController
from scipy.spatial.transform import Rotation as R

In [77]:
# Sim Constants
DT_S = 1. / 60

DRONE_MASS = 0.027
NEWTON_TO_RPM = 0.2

Ixx = 1.4e-5
Iyy = 1.4e-5
Izz = 2.17e-5

MIN_THRUST = 0.26477955 / 4 * 0.8
MAX_THRUST = 0.26477955 / 4 * 1.5

In [ ]:
def reset():
    env.reset()

In [79]:
env.get_obs()

In [80]:
env.step([0, 0, 0, 0])

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
def sim_step(motors):
    # change motor order
    motors = np.array([motors[0], motors[2], motors[3], motors[1]])

    # scale
    motors = motors * NEWTON_TO_RPM

    result = env.step(action=motors.tolist())[0]

    position = np.array([
        result['position'][0],
        result['position'][2],
        result['position'][1],
    ])
    velocity = np.array([
        result['velocity'][0],
        result['velocity'][2],
        result['velocity'][1],
    ])
    attitude = R.from_quat([
        result['attitude'][0],
        result['attitude'][1],
        result['attitude'][1],
        result['attitude'][3]],
        scalar_first=False
    )
    omega = np.array([
        result['omega'][0],
        result['omega'][2],
        result['omega'][1],
    ])

    omega[0], omega[1], omega[2] = -omega[0], -omega[1], omega[2]

    return {
        'position': position,
        'velocity': velocity,
        'attitude': attitude,
        'omega': omega
    }

In [ ]:
quadcopter_controller = QuadcopterController(
    DT_S,
    DRONE_MASS * 0.99,
    Ixx, Iyy, Izz,
    0.7,
    12, 5, 5,
    0.03,
    MIN_THRUST, MAX_THRUST,
)

In [ ]:
def tune_controller(
        target={
            "position": np.array([0, 0, 5]),
            "velocity": np.zeros(3),
            "acceleration": np.zeros(3),
            "attitude": R.from_quat([0, 0, 0, 1]),
        },
        kp_pqr=np.array([95.0, 95.0, 6.0]),
        kp_bank=0.0,
        kp_pos_z=0.0,
        kp_vel_z=0.0,
        ki_pos_z=0.0,
        kp_pos_xy=0.0,
        kp_yaw=0.0,
        kp_vel_xy=0.0,
        kappa=1.0,
        n_episodes=500,
):
    reset()

    quadcopter_controller.integrated_altitude_error = 0.0
    quadcopter_controller.set_gains(
        kp_pqr=kp_pqr,
        kp_bank=kp_bank,
        kp_pos_z=kp_pos_z,
        kp_vel_z=kp_vel_z,
        ki_pos_z=ki_pos_z,
        kp_pos_xy=kp_pos_xy,
        kp_yaw=kp_yaw,
        kp_vel_xy=kp_vel_xy,
        kappa=kappa
    )

    times = []
    positions = []
    attitudes = []
    motors_ = []
    omegas = []
    velocities = []

    state = sim_step([0, 0, 0, 0])

    t = 0
    for i in tqdm(range(n_episodes)):

        motors = quadcopter_controller.run_control(
            target['position'],
            target['velocity'],
            target['acceleration'],
            target['attitude'],
            state['position'],
            state['velocity'],
            state['omega'],
            state['attitude'],
        )
        motors_.append(motors)

        # accelerations.append(state['acceleration'])
        attitudes.append(state['attitude'].as_quat())
        positions.append(state['position'])
        omegas.append(state['omega'])
        velocities.append(state['velocity'])
        times.append(t)
        state = sim_step(motors)

        t += DT_S

    return (
        np.array(times),
        np.array(positions),
        np.array(velocities),
        np.array(attitudes),
        np.array(omegas),
        np.array(motors_)
    )

In [ ]:
reset()
target = {
    "position": np.array([3, 5, 10]),
    "velocity": np.zeros(3),
    "acceleration": np.zeros(3),
    "attitude": R.from_quat([0, 0, 0, 1]),
}

In [ ]:
(times, positions, velocities, attitudes, omegas, motors) =  tune_controller(
    kp_pqr=np.array([25, 25, 5]),
    ki_pos_z=2,
    kp_pos_z=6,
    kp_vel_z=12,
    kp_pos_xy=8.0,
    kp_vel_xy=8.0,
    kp_yaw=1.0,
    kp_bank=2.0,
    n_episodes=500,
    target=target
)

In [ ]:
plt.title("z")
plt.plot(positions[:, 2])
plt.show()

plt.title("x")
plt.plot(positions[:, 0])
plt.show()

plt.title("y")
plt.plot(positions[:, 1])
plt.show()